In [51]:
import polars as pl
import pandas as pd
import plotly.express as px

def plot_trade_distribution_with_offset(file_path: str):
    # load
    df = pl.read_csv(file_path)

    # Convertir 'xltime' en datetime tout en prenant en compte l'offset horaire (en local, pas UTC)
    df = df.with_columns(
        pl.col("xltime").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f%:z").alias("datetime")
    )

    #Convert en Pandas 
    df_pandas = df.to_pandas()
    
    # Convertir le datetime en heure locale
    df_pandas['datetime'] = df_pandas['datetime'].dt.tz_convert('America/New_York')  # Set the correct timezone (-05:00)

    # Extraire l'heure locale
    df_pandas['hour'] = df_pandas['datetime'].dt.hour

    # Filtrer les heures entre 9h et 23h59 (de 9h AM jusqu'à minuit)
    filtered_hours = df_pandas[(df_pandas['hour'] >= 9) & (df_pandas['hour'] < 24)]['hour']

    # Créer le plot
    fig = px.histogram(
        df_pandas['hour'],
        x="hour",
        nbins=15,
        title="Distribution Temporelle des Trades (9h - 00h)",
        labels={"hour": "Heure de la Journée"},
        range_x=[9, 24],  # Plage de l'axe des X de 9h à 00h (minuit)
        color_discrete_sequence=['blue']
    )

    # pimp my graph
    fig.update_layout(bargap=0.2, template="plotly_white")
    fig.show()

# Utilisation
#file_path = "data_SPX.csv"  # Remplacez avec votre fichier CSV
#plot_trade_distribution_with_offset(file_path)

